In [1]:
p = "/kaggle/input/ai-in-healthcare-hackathon/aroi"

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import keras.backend as K
from keras.layers import *
from keras.models import *
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import cv2
from tensorflow import keras
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img

## Data preprocessing

In [3]:
img_path = []
mask_path = []

ip = os.path.join(p,'images')
mp = os.path.join(p,'masks')

for i, j in zip(os.listdir(ip),os.listdir(mp)):
    img_path.append(os.path.join(ip, i))
    mask_path.append(os.path.join(mp, j))

In [4]:
img_path = sorted(img_path)
mask_path = sorted(mask_path)

In [ ]:
# df = pd.DataFrame({'Images':img_path,'Masks':mask_path})
# df.head()
# df_train, df_test = train_test_split(df,test_size = 0.2, random_state = 42)
# print("Train Set : {0}\nTest Set : {1}".format(df_train.values.shape[0],df_test.values.shape[0])) 

In [6]:
X_train, X_test, y_train, y_test = train_test_split(img_path,mask_path,test_size = 0.2, random_state = 42)
len(X_train), len(X_test) 

(884, 221)

## Data pipeline

In [9]:
class make_data(keras.utils.Sequence):
    
    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths
        
    def __len__(self):
        return len(self.target_img_paths) // self.batch_size
    
    def __getitem__(self, idx):
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=self.img_size,color_mode="rgb")
#             input_arr = tf.keras.preprocessing.image.img_to_array(img)
#             input_arr = np.array([input_arr])
#             input_arr = input_arr/255
            x[j] = img
        
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")
        for j, path in enumerate(batch_target_img_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")
#             input_arr = tf.keras.preprocessing.image.img_to_array(img)
#             input_arr = np.array([input_arr])
            y[j] = np.expand_dims(img, 2)
            
        return x/255,y
train_gen = make_data(batch_size = 32, img_size = (256,256), input_img_paths =X_train, target_img_paths=y_train)
test_gen = make_data(batch_size = 32, img_size = (256,256),input_img_paths = X_test,target_img_paths = y_test)    

In [ ]:
# class make_data(keras.utils.Sequence):
    
#     def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
#         self.batch_size = batch_size
#         self.img_size = img_size
#         self.input_img_paths = input_img_paths
#         self.target_img_paths = target_img_paths
        
#     def __len__(self):
#         return len(self.target_img_paths) // self.batch_size
    
#     def __getitem__(self, idx):
#         i = idx * self.batch_size
#         batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
#         batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
#         x = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="float32")
#         for j, path in enumerate(batch_input_img_paths):
#             img = Image.open(path)
#             img = img.resize((256,512))
#             x[j] = np.expand_dims(img, 2)
        
#         y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")
#         for j, path in enumerate(batch_target_img_paths):
#             img = Image.open(path)
#             img = img.resize((256,512))
#             y[j] = np.expand_dims(img, 2)
            
#         return x/255,y
# train_gen = make_data(batch_size = 32, img_size = (512,256), input_img_paths =X_train, target_img_paths=y_train)
# test_gen = make_data(batch_size = 32, img_size = (512,256),input_img_paths = X_test,target_img_paths = y_test)    

In [ ]:
# w = 256
# h = 512
# def data_generator(df,augmentation):
#     image_generator = augmentation.flow_from_dataframe(
#       df,
#       x_col = "Images",
#       target_size=(h,w),
#       color_mode="rgba",
#       batch_size=32,
#       seed=1,
#       class_mode= None
#     )

#     mask_generator = augmentation.flow_from_dataframe(
#       df,
#       x_col = "Masks",
#       target_size=(h,w),
#       color_mode="grayscale",
#       batch_size=32,
#       seed=1,
#       class_mode = "sparse",
#     )

#     gen = zip(image_generator, mask_generator)
#     for image, mask in gen:
#         yield image,mask

# train_aug = ImageDataGenerator(
#     rescale = 1./255
#     )

# test_aug = ImageDataGenerator()

# train_gen1 = data_generator(df_train,train_aug)
# test_gen1 = data_generator(df_test,test_aug) 

In [8]:
def iou(y_true, y_pred):
    smooth = 1.
    intersection = K.sum(y_true * y_pred)
    summ = K.sum(y_true + y_pred)
    iou = (intersection + smooth) / (summ - intersection + smooth)
    return iou

## Model building and compiling

In [10]:
def Unet():
    inp = Input((256,256, 3))
    conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inp)
    conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(7, 1, activation = 'softmax')(conv9)

    model = Model(inputs = inp, outputs = conv10)
    return model

In [11]:
checkpointer = ModelCheckpoint(filepath="model.hdf5", 
                               verbose=1, 
                               save_best_only=True
                              )
earlystopping = EarlyStopping(monitor='val_loss',
                              mode='auto', 
                              verbose=1, 
                              patience=13
                             )
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              mode='auto',
                              verbose=1,
                              patience=6,
                              min_delta=0.0001,
                              factor=0.2
                             )

In [12]:
unet = Unet()
unet.compile(optimizer = 'adam', loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics = [iou])

## Model training

In [13]:
history = unet.fit(train_gen ,epochs = 100, validation_data = test_gen, callbacks = [checkpointer, earlystopping, reduce_lr])

Epoch 1/100
27/27 [==============================] - 25s 635ms/step - loss: nan - iou: nan - val_loss: nan - val_iou: nan

Epoch 00001: val_loss did not improve from inf
Epoch 2/100
27/27 [==============================] - 15s 531ms/step - loss: nan - iou: nan - val_loss: nan - val_iou: nan

Epoch 00002: val_loss did not improve from inf
Epoch 3/100
26/27 [===========================>..] - ETA: 0s - loss: nan - iou: nan

KeyboardInterrupt: 